# МИИАД проект. Часть 2
В этом блокноте будем работать с эмбедингами музыкальных произведений из [датасета](https://github.com/mdeff/fma), которые были получены с помощью [VGGish](https://github.com/tensorflow/models/tree/master/research/audioset/vggish).  

In [ ]:
! unzip embeddings.zip

In [11]:
import torch
import pandas as pd
import numpy as np 
import os, sys

import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm.notebook import tqdm

%matplotlib inline

In [86]:
data_train.songs.loc[2].drop('genre_top').to_numpy(dtype=np.float32).shape

(186,)

In [176]:
class MusicDataset(torch.utils.data.Dataset):
    """Music dataset."""
 
    def __init__(self, csv_file, root_dir, select):

        self.songs = pd.read_csv(csv_file).set_index('track_id')
        self.classes = {}
        for ind, genre in enumerate(self.songs['genre_top'].unique()):
            self.classes[genre] = ind
        self.root_dir = root_dir
        with open(select, 'rb') as f:
            self.select = np.load(f)
 
    def __len__(self):
        return len(self.select)
 
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        ind = self.select[idx]
        song_name = os.path.join(self.root_dir,
                                 str(ind) + '.npy')
        with open(song_name, 'rb') as f:
            song = np.load(f)
        target = self.songs.loc[ind]['genre_top']
        # data = self.songs.loc[ind].drop('genre_top').to_numpy(dtype=np.float32)
        sample = {'song': song.flatten(), 'target': self.classes[target]}
 
        return sample

In [177]:
data_train = MusicDataset('data.csv', 'embeddings', 'train.npy')
data_test = MusicDataset('data.csv', 'embeddings', 'test.npy')
data_val = MusicDataset('data.csv', 'embeddings', 'val.npy')
torch.manual_seed(42)

In [178]:
data_train[0]['song'].shape

(3968,)

In [179]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=10, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=10, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=10, shuffle=True)

In [180]:
class FullNN(nn.Module):
    def __init__(self):
        super(FullNN, self).__init__()
        self.fc1 = nn.Linear(31*128, 128)
        self.fc2 = nn.Linear(128, 60)
        self.fc3 = nn.Linear(60, 8)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

model = FullNN()

In [71]:
for batch in train_dataloader:
    with torch.no_grad():
        model1 = model.to('cuda:0').eval()
        labels = batch['target'].to('cuda:0')
        song = batch['song'].to('cuda:0')
        predicted = model1(song)
        print((predicted.argmax(1) == labels).float().mean())
        break

tensor(0.2400, device='cuda:0')


In [181]:
def train_epoch(
    model,
    data_loader,
    optimizer,
    criterion,
    return_losses=False,
    device='cuda:0',
):
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])
    total_labels = np.array([])
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for batch in data_loader:
            # Move Batch to GPU
            songs = batch['song'].to(device)
            labels = batch['target'].to(device)
            predicted = model(songs)
            loss = criterion(predicted, labels)
            # Update weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            # Update descirption for tqdm
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())
    metrics = {'loss': total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    if return_losses:
        return metrics, all_losses
    else:
        return metrics


def validate(model, data_loader, criterion, device='cuda:0'):
    model = model.eval()
    total_loss = 0
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for batch in data_loader:
            songs = batch['song'].to(device)
            labels = batch['target'].to(device)
            predicted = model(songs)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
    metrics = {'loss': total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    return metrics

def fit(
    model,
    epochs,
    train_data_loader,
    validation_data_loader,
    optimizer,
    criterion
):
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    epoch_train_acc = []
    epoch_eval_acc = []
    for epoch in range(epochs):
        # Train step
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_epoch(
            model=model,
            data_loader=train_data_loader,
            optimizer=optimizer,
            return_losses=True,
            criterion=criterion
        )
        # Save Train losses
        all_train_losses.extend(one_epoch_train_losses)
        epoch_train_losses.append(train_metrics['loss'])
        epoch_train_acc.append(train_metrics['accuracy'])
        # Eval step
        print(f"Validation Epoch: {epoch}")
        with torch.no_grad():
            validation_metrics = validate(
                model=model,
                data_loader=validation_data_loader,
                criterion=criterion
            )
        # Save eval losses
        epoch_eval_losses.append(validation_metrics['loss'])
        epoch_eval_acc.append(validation_metrics['accuracy'])

        plt.figure()
        plt.subplot(121)
        plt.plot(epoch_eval_acc, 'r--')
        plt.plot(epoch_train_acc, 'b--')

        plt.subplot(122)
        plt.plot(range(epoch+1), epoch_eval_losses, 'r--', range(epoch+1), epoch_train_losses, 'b--')
        plt.show()

In [182]:
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005) 

In [ ]:
fit(model, 1000, train_dataloader, val_dataloader, optimizer, criterion)

In [185]:
with torch.no_grad():
            validation_metrics = validate(
                model=model,
                data_loader=test_dataloader,
                criterion=criterion
            )


In [186]:
validation_metrics

{'accuracy': 0.21875, 'loss': 2.0265617325901983}